<a href="https://colab.research.google.com/github/anita-maxwynn/Crypto-shit/blob/main/Diffie%E2%80%93Hellman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Diffie-Hellman

In [ ]:
!pip install cryptography

In [ ]:
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.backends import default_backend

In [ ]:
parameters=dh.generate_parameters(generator=2,key_size=2048,backend=default_backend())

In [ ]:
print(parameters)

In [ ]:
private_key_1=parameters.generate_private_key()
public_key_1 = private_key_1.public_key()

print(private_key_1)
print(public_key_1)

In [ ]:
private_key_2=parameters.generate_private_key()
public_key_2 = private_key_2.public_key()

print(private_key_2)
print(public_key_2)

In [ ]:
x=public_key_1
y=public_key_2

In [ ]:
shared_key_1=private_key_1.exchange(y)
shared_key_2=private_key_2.exchange(x)

In [ ]:
if shared_key_1==shared_key_2:
  print("Shared keys are equal")
else:
  print("Shared keys are not equal")

Shared keys are equal


Mathematically:

* A’s private = `a`
* A’s public = `g^a mod p`
* B’s private = `b`
* B’s public = `g^b mod p`

When A computes:

```
shared_key_1 = (g^b)^a mod p = g^(ab) mod p
```

When B computes:

```
shared_key_2 = (g^a)^b mod p = g^(ab) mod p
```

They are **identical** (because multiplication is commutative: `ab = ba`).



# Task
Encrypt a message using a shared secret key and verify the encryption by decrypting the message.

## Key derivation

### Subtask:
Derive a symmetric encryption key from the shared secret key using a suitable Key Derivation Function (KDF).


**Reasoning**:
Import necessary modules for key derivation and define a salt. Then, instantiate the KDF and derive the symmetric key from the shared secret.



In [ ]:
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend

# Define a salt
salt = b"thisisagoodsaltformyproject"

# Instantiate PBKDF2HMAC
kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=salt,
    iterations=100000,
    backend=default_backend()
)

# Derive the key
derived_key = kdf.derive(shared_key_1)

print(derived_key)

b'\x8f\x9dCU)\xba\xc0\xdb;\xa5\xecyoJ\xf2\x92\x15\xc7I%5p}8\x13\xb0e\xa0\x07\xfe\xb0e'


## Encryption

### Subtask:
Encrypt a sample message using the derived key and a symmetric encryption algorithm (e.g., AES).


**Reasoning**:
Encrypt a sample message using the derived key and AES in CBC mode.



In [ ]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import os
from cryptography.hazmat.primitives import padding

# Define a sample message
message = b"This is a secret message that needs to be encrypted."

# Generate a random initialization vector (IV)
iv = os.urandom(16)  # AES block size is 16 bytes

# Create an AES encryptor object
cipher = Cipher(algorithms.AES(derived_key), modes.CBC(iv), backend=default_backend())
encryptor = cipher.encryptor()

# Apply PKCS7 padding to the message
padder = padding.PKCS7(algorithms.AES.block_size).padder()
padded_data = padder.update(message) + padder.finalize()

# Encrypt the padded message
ciphertext = encryptor.update(padded_data) + encryptor.finalize()

# Print the original message, the IV, and the ciphertext
print("Original Message:", message)
print("IV:", iv)
print("Ciphertext:", ciphertext)

Original Message: b'This is a secret message that needs to be encrypted.'
IV: b'\xa2\xc5\x7f\xb4\x9b*m\xc5SQ\x04\x9f\xdan\xea\x87'
Ciphertext: b'\x92\xfb\x91\xff5\x90\xae\x8b\x12\x95*[\xa5\xdd\x96J=("4\xde\xf5\xd6.sy3\x07\xe1&\xb75\x93\xceb\x82\x16!\xb7Z\xae\xef\x81J\xa0\xbf\xab\xb6\xdd\xac\xd2\xc5\x07\x80\x82\\\xd1\x81\x1f\xe2[d\x8eF'


**Reasoning**:
The previous attempt failed because the message length is not a multiple of the block size. Padding is required for CBC mode.



In [ ]:
from cryptography.hazmat.primitives import padding

# Define a sample message
message = b"This is a secret message that needs to be encrypted."

# Generate a random initialization vector (IV)
iv = os.urandom(16)  # AES block size is 16 bytes

# Create an AES encryptor object
cipher = Cipher(algorithms.AES(derived_key), modes.CBC(iv), backend=default_backend())
encryptor = cipher.encryptor()

# Apply PKCS7 padding to the message
padder = padding.PKCS7(algorithms.AES.block_size).padder()
padded_data = padder.update(message) + padder.finalize()

# Encrypt the padded message
ciphertext = encryptor.update(padded_data) + encryptor.finalize()

# Print the original message, the IV, and the ciphertext
print("Original Message:", message)
print("IV:", iv)
print("Ciphertext:", ciphertext)

Original Message: b'This is a secret message that needs to be encrypted.'
IV: b'\x83@\xfe\xc81{f\x10\xb3\xdct\xc9\xcd\x9c\xf8\xcf'
Ciphertext: b'\xfe\xe1a\xbc\x11\xb5,\x1b\xcdq=\x1bw@W\xad\xc6\xcc\xef\xf3\xce\xf6\xd5\xc6\x85\xa7\x9es\xca\x96Nm\x9bq\xddNW\xea\xe9g\x02=\x96\xc9[\xbb\xfcY\xa6\x04\xd3K=\xad\xb9\x89\x0b\xd4\x1b\x04s*\xcb\x15'


## Decryption

### Subtask:
Decrypt the encrypted message using the same derived key and symmetric encryption algorithm.


**Reasoning**:
Decrypt the encrypted message using the derived key, the same IV, and the CBC mode, applying PKCS7 unpadding.



In [ ]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding

# Create an AES decryptor object
cipher = Cipher(algorithms.AES(derived_key), modes.CBC(iv), backend=default_backend())
decryptor = cipher.decryptor()

# Decrypt the ciphertext
decrypted_padded_data = decryptor.update(ciphertext) + decryptor.finalize()

# Unpad the decrypted data
unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
decrypted_message = unpadder.update(decrypted_padded_data) + unpadder.finalize()

# Print the decrypted message
print("Decrypted Message:", decrypted_message)

Decrypted Message: b'This is a secret message that needs to be encrypted.'


## Verification

### Subtask:
Verify that the decrypted message is the same as the original message.


**Reasoning**:
Compare the decrypted message with the original message and print the result.



In [ ]:
if decrypted_message == message:
  print("Decryption successful: The decrypted message matches the original message.")
else:
  print("Decryption failed: The decrypted message does not match the original message.")

Decryption successful: The decrypted message matches the original message.


## Summary:

### Data Analysis Key Findings

*   A symmetric encryption key was successfully derived from a shared secret key using PBKDF2HMAC with SHA256, a key length of 32 bytes, a specified salt, and 100,000 iterations.
*   The sample message "This is a secret message that needs to be encrypted." was successfully encrypted using AES in CBC mode with a randomly generated 16-byte IV and PKCS7 padding to handle the message length.
*   The ciphertext was successfully decrypted back to the original message using the same derived key, IV, and decryption/unpadding process.
*   The decrypted message was verified to be identical to the original message, confirming the success of the encryption and decryption process.

### Insights or Next Steps

*   The successful encryption and decryption demonstrate the correct implementation of a basic symmetric encryption scheme using derived keys.
*   For production environments, consider storing the salt and iteration count securely and using a more robust method for handling the shared secret key.
